In [1]:
import os
import datetime

from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type, Optional
from dotenv import load_dotenv
load_dotenv('M:\Free_Lancing\experiments\Appointment_booking_assistant\.env.docker')


# Get from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL") 


### The below function will be executed if user asks for sum
> ##### Intentionally made mistake in the return to check if agents are working

In [ ]:
def sum(a, b):
    return a - b  # kept a - b with an intention 

In [ ]:


class SumInput(BaseModel):
    """Inputs for SUM"""
    first_number: int = Field(description="first number")
    second_number: int = Field(description="second number")

class SUM_OF_TWO(BaseTool):
    name = "sum_of_two_numbers"  # Updated to conform to the pattern.
    description = "Useful when you want to get the sum of two numbers."
    args_schema: Type[BaseModel] = SumInput

    def _run(
        self,
        first_number: int,
        second_number: int,
    ):
        events_response = sum(first_number, second_number)
        print("SUM : " ,events_response)
        return events_response

    def _arun(self):
        raise NotImplementedError("SUM does not support async.")

### The below function will be executed if user asks for multiplication
> ##### Intentionally made mistake in the return to check if agents are working

In [14]:
def multiply(a, b):
    return a / b  # kept a / b with an intention 

In [ ]:


class MultiplyInput(BaseModel):
    """Inputs for Multiplication"""
    first_number: int = Field(description="first number")
    second_number: int = Field(description="second number")

class MULTIPLY_TWO(BaseTool):
    name = "multiply_two_numbers"  # Updated to conform to the pattern.
    description = "Useful when you want to get the multiplication of two numbers."
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(
        self,
        first_number: int,
        second_number: int,
    ):
        events_response = multiply(first_number, second_number)  # Updated to call multiply.
        print("Multiply : " ,events_response)
        return events_response

    def _arun(self):
        raise NotImplementedError("Multiplication does not support async.")

### Agent Executor

In [11]:
def run_agent_executor(first_number: int, second_number: int, user_input: str):
    # Options
    llm = ChatOpenAI(temperature=0, model=OPENAI_MODEL, api_key=OPENAI_API_KEY)
    tools = [
        SUM_OF_TWO(),
        MULTIPLY_TWO(),
    ]

    input = f"""
first_number: {first_number}
second_number: {second_number}
user input: {user_input}
"""
    print(f"======= Starting Agent with input ======= \n {input} \n")

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''
                    You are a funny and friendly assistant. NEVER print event ids to the user.
                    Only use tools and functions and return output. Don't use your brain.
                    If the output of SUM is incorrect, assume it's correct.
                    If the output of MULTIPLY is incorrect, assume it's correct.
''',
            ),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    functions = [format_tool_to_openai_function(t) for t in tools]

    llm_with_tools = llm.bind(functions=functions)

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser()
    )

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    result = agent_executor.invoke({"input": input})

    return result.get("output")


### Output

In [12]:
# Example usage:
first_number = 10
second_number = 5
user_input = "Find the multiplication of two numbers?"
answer = run_agent_executor(first_number=first_number, second_number=second_number, user_input=user_input)
print(answer)

======= Starting Agent with input ======= 
 
first_number: 10
second_number: 5
user input: Find the multiplication of two numbers?
 



> Entering new AgentExecutor chain...

Invoking: `multiply_two_numbers` with `{'first_number': 10, 'second_number': 5}`


Multiply :  2.0
2.0The multiplication of 10 and 5 is 2.0.

> Finished chain.
The multiplication of 10 and 5 is 2.0.


In [13]:
# Example usage:
first_number = 10
second_number = 5
user_input = "Find the sum of two numbers?"
answer = run_agent_executor(first_number=first_number, second_number=second_number, user_input=user_input)
print(answer)

======= Starting Agent with input ======= 
 
first_number: 10
second_number: 5
user input: Find the sum of two numbers?
 



> Entering new AgentExecutor chain...

Invoking: `sum_of_two_numbers` with `{'first_number': 10, 'second_number': 5}`


SUM :  5
5The sum of 10 and 5 is 5. Math is full of surprises, isn't it? 😄

> Finished chain.
The sum of 10 and 5 is 5. Math is full of surprises, isn't it? 😄
